# Primzahltests

In [1]:
import pandas as pd

## Fermat-Test
#### Lösung Aufgabe 1

In [2]:
def fermat_test(n, basen=range(2,21)):
    for a in basen:
        potenz = pow(a, n - 1, n)
        res = "Möglicherweise prim" if potenz == 1 else "Sicher zusammengesetzt"
        print(f"{a:3}**{n-1}={potenz:8}: {res}")

In [3]:
fermat_test(29341)

  2**29340=       1: Möglicherweise prim
  3**29340=       1: Möglicherweise prim
  4**29340=       1: Möglicherweise prim
  5**29340=       1: Möglicherweise prim
  6**29340=       1: Möglicherweise prim
  7**29340=       1: Möglicherweise prim
  8**29340=       1: Möglicherweise prim
  9**29340=       1: Möglicherweise prim
 10**29340=       1: Möglicherweise prim
 11**29340=       1: Möglicherweise prim
 12**29340=       1: Möglicherweise prim
 13**29340=   18057: Sicher zusammengesetzt
 14**29340=       1: Möglicherweise prim
 15**29340=       1: Möglicherweise prim
 16**29340=       1: Möglicherweise prim
 17**29340=       1: Möglicherweise prim
 18**29340=       1: Möglicherweise prim
 19**29340=       1: Möglicherweise prim
 20**29340=       1: Möglicherweise prim


## Miller-Rabin-Test
#### Lösung Aufgabe 3

In [4]:
def miller_rabin(n, basen=[2, 3, 5, 7, 11]):
    """
    Primzahltest nach Miller-Rabin.
    :param n: Zahl, die getestet werden muss.
    :param basen: Die Basiszahlen für die Potenzen im Test.
          Default=[2, 3, 5, 7, 11]
    :return: Ein Dataframe mit den berechneten Potenzen. 
             Die letzte Spalte gibt an, 
             ob eine Basiszahl ein MR-Zeuge für Primzahl ist.
             Wenn die Letzte Spalte nur aus True besteht, wahrscheinlich prim;
             sonst sicher zusammengesetzt.
    """
    m = n - 1
    zweier = 0
    while m % 2 == 0:
        m //= 2
        zweier += 1
    resultate = []
    daten = []
    for a in basen:
        row = [pow(a, m, n)]
        for i in range(zweier):
            row.append(pow(row[-1], 2, n)) 
        daten.append(row)
        resultate.append(test_mr_row(row, n - 1))
    # return basen, daten, resultate
    # Pandas wird nur verwendet wegen der schönen Formatierung
    return als_dataframe(basen, m, daten, resultate)

def test_mr_row(row, minus_1):
    """
    Wertet eine Zeile im Miller-Rabintest aus.
    :param row: Auszuwertende Zeile.
    :param minus_1: Die Zahl n-1 für die auf Primalität zu testende Zahl.
    :returns: True, wenn Zeile besagt, dass Basis eine Primzahl sein könnte,
              False, wenn es sich sicher um keine Primzahl handelt.
    """
    if row[-1] != 1:
        return False
    if row[0] == 1:
        return True
    return row[list(row).index(1) -1] == minus_1 

def als_dataframe(basen, m, daten, resultate, 
                  true="Möglicherweise prim", 
                  false="Sicher zusammengesetzt"):
    """
    Packt die Argumente in ein DataFrame.
    """
    spalten = [m * 2**i for i in range(len(daten[0]))]
    df = pd.DataFrame(daten, index=basen, columns=spalten)
    resultate = [true if res else false for res in resultate]
    df["Res"] = resultate
    return df

#### Lösungen Aufgabe 2

In [5]:
miller_rabin(101)

,25,50,100,Res
2,10,100,1,Möglicherweise prim
3,10,100,1,Möglicherweise prim
5,1,1,1,Möglicherweise prim
7,10,100,1,Möglicherweise prim
11,10,100,1,Möglicherweise prim


In [6]:
miller_rabin(10201)

,1275,2550,5100,10200,Res
2,8272,7877,4647,9293,Sicher zusammengesetzt
3,1505,403,9394,8586,Sicher zusammengesetzt
5,5859,1516,3031,6061,Sicher zusammengesetzt
7,1606,8584,3233,6465,Sicher zusammengesetzt
11,5444,3231,3738,7475,Sicher zusammengesetzt


In [7]:
miller_rabin(29341)

,7335,14670,29340,Res
2,26424,29340,1,Möglicherweise prim
3,22569,1,1,Sicher zusammengesetzt
5,15127,25011,1,Sicher zusammengesetzt
7,23496,11101,1,Sicher zusammengesetzt
11,1331,11101,1,Sicher zusammengesetzt


Im letzten Beispiel sieht man, dass der Miller-Rabin-Test dem Fermat-Test überlegen ist: Der Fermat-Test würde nur die Spalte mit dem Namen 29340 betrachten, und könnte daraus nicht schließen, dass 29341 keine Primzahl ist.

In [8]:
miller_rabin(5)

,1,2,4,Res
2,2,4,1,Möglicherweise prim
3,3,4,1,Möglicherweise prim
5,0,0,0,Sicher zusammengesetzt
7,2,4,1,Möglicherweise prim
11,1,1,1,Möglicherweise prim


# RSA
#### Aufgabe 7

In [9]:
def erzeuge_rsa_schluesselpaar(p, q, e = 3):
    """
    Erzeugt ein RSA-Schlüsselpaar aus zwei Primzahlen.
    Die Parameter werden nicht verifiziert.
    
    :param p: Eine (große) Primzahl.
    :param q: Eine andere (große) Primzahl.
    :param e: Vorschlag für Exponenten im öffentlichen Schlüssel.
              Wenn das nicht geht, wird e vergrößert.
    :return: Ein Paar, bestehend aus 
             dem öffentlichem und privatem Schlüsselpaar.
    """
    n = p * q
    phi = (p-1) * (q - 1)
    while phi % e == 0:
        e += 1
    d = pow(e, -1, phi)
    return (n, e), (n, d)

In [10]:
def rsa_anwenden(nachricht, schluessel):
    """
    Verschlüsselt oder entschlüsselt eine Nachricht mit einem RSA-Schlüssel.
    
    :param nachricht: Zu verschlüsselnde oder zu entschlüsselde Nachricht.
    :param schluessel: Ein öffentlicher oder privater RSA-Schlüssel.
    """
    return pow(nachricht, schluessel[1], schluessel[0])

### Beispiele zu RSA

#### Aufgabe 4
Sie möchten sich mit dem RSA – Verfahren verschlüsselte Nachrichten senden lassen.
Dazu wählen Sie die Primzahlen $p=7$, $q=11$ und die Zahl $e=13$.

In [11]:
# Erzeugen der Schluessel
oeff, priv = erzeuge_rsa_schluesselpaar(7, 11, e=13)
print(oeff)
print(priv)

(77, 13)
(77, 37)


In [12]:
# Entschlüsseln der Nachricht 6
rsa_anwenden(6, priv)

41

In [13]:
# Test
rsa_anwenden(41, oeff)

6

#### Aufgabe 5
Wählen Sie jetzt $p=7$, $q=9$ und $e=11$
für die RSA – Verschlüsselung.

1. Bestimmen Sie wiederum den öffentlichen Schlüssel.
2. Verschlüsseln Sie damit die Nachricht $x = 3$.
3. Entschlüsseln Sie die Nachricht wieder.

In [14]:
# Erzeugen der Schluessel
oeff, priv = erzeuge_rsa_schluesselpaar(7, 9, e=11)
print(oeff)
print(priv)

(63, 11)
(63, 35)


In [15]:
# Verschlüsseln der Nachricht 3
rsa_anwenden(3, oeff)

54

In [16]:
# Entschlüsseln der Nachricht 54
rsa_anwenden(54, priv)

45

Das Resultat ist falsch, die wieder entschlüsselte Nachricht ist verschieden von der ursprünglichen Nachricht!   
Der Grund ist, dass 9 keine Primzahl ist!   
Vielleicht sollte die Funktion `erzeuge_rsa_schluesselpaar` die Argumente doch validieren!  ;-)

# Diffie-Hellman
#### Aufgabe 8

In [17]:
p = 109

Es soll eine Basiszahl g bestimmt werden, so dass es $p-1$ viele modulare Potenzen von $g$ gibt.

In [18]:
def finde_generator(p):
    """
    Findet ein Generatorelement für Z_p.
    :param p: Primzahl
    :returns: Das kleineste Generatorelement für Z_p.
    """
    for g in range(2, p):
        if len({pow(g, i, p) for i in range(p)}) == p - 1:
            return g

In [19]:
g = finde_generator(p)
g

6

Damit haben wir geeignetes $g$ gefunden.   
Es gibt effizientere Methoden als unsere Funktion `finde_generator`, ein solches $g$ zu finden, das ist aber nicht Thema dieses Moduls.   
Wenn die Anzahl der Potenzen nicht gleich $p-1$ ist, aber trotzdem groß, so ist das auch in Ordnung.

In [20]:
# Schlüsselaustausch
# Alice wählt Zufallszahl x und sendet $a = g^x \% p$ an Bob
x = 30
a = pow(g, x, p)
a

43

In [21]:
# Bob wählt Zufallszahl y und sendet $b = g**y % p$ an Alice
y = 42
b = pow(g, y, p)
b

34

In [22]:
# Bob berechnet den Schlüssel a**y % p
pow(a, y, p)

45

In [23]:
# Alice berechnet den Schlüssel b^x % p
pow(b, x, p)

45

Alice und Bob kennen jetzt einen gemeinsamen Schlüssel.   
Wenn Eve mitgehört hat, nützt ihr das nichts, denn sie kennt weder $x$ noch $y$.   

# ElGamal
#### Aufgabe 11

In [24]:
# secrets ist ein Modul für kryptographisch sichere Zufallszahlen.
# Das Modul random ist für Simulationen geeignet,
# ist kryptopgraphisch aber nicht sicher.
import secrets

In [25]:
def erzeuge_elgamal_schluesselpaar(p, g, x=None):
    """
    Erzeugt ein Paar, bestehnd aus einem öffentlichen und privatem ElGamal-schlüssel.
    :param p: Eine Primzahl
    :param g: Ein Generatorelement für Z_p
    :param x: Eine Zufallszahl für denprivaten Schlüssel.
              Wenn sie nicht angegeben wird, wird sie zufällig erzeugt.
    """
    if x is None:
        x = 1 + secrets.randbelow(p - 2)
    g_hoch_x = pow(g, x, p)
    oeffentlich = (p, g, g_hoch_x)
    privat      = (p, g, x)
    return oeffentlich, privat

In [26]:
def elgamal_verschluesseln(nachricht, oeffentlicher_schluessel, y=None):
    """
    Verschlüsselt eine Nachricht mit ElGamal.
    :param nachricht: zu verschlüsselnde Nachricht.
    :param oeffentlicher_schluessel: Ein öffentlicher ElGamal-Schlüssel als Tripel.
    :param y: Zufallszahl, die zum verschlüsseln verwendet wird.
              Falls diese weggelassen wird, wird sie zufällig bestimmt.
    :returns: Ein Chiffrat als Paar
    """
    p, g, g_hoch_x = oeffentlicher_schluessel
    if y is None:
        y = 1 + secrets.randbelow(p - 2)
    g_hoch_y = pow(g, y, p)
    c = pow(g_hoch_x, y, p) * nachricht % p
    return (g_hoch_y, c)

In [27]:
def elgamal_entschluesseln(chiffrat, privater_schluessel):
    """
    Entschlüsselt eine chiffrierte Nachricht mit ElGamal.
    :param chiffrat: Verschlüsselte Nachricht als Paar.
    :param privater_schluessel: Ein privater ElGamal-Schlüssel als Tripel.
    :returns: Die entschlüsselte Nachricht.
    """
    g_hoch_y, c = chiffrat
    p, g, x = privater_schluessel
    return pow(g_hoch_y, p - 1 - x, p) * c % p

#### Aufgabe 9

In [28]:
p = 23

In [29]:
# Suche eines Generators g
g = finde_generator(p)
g

5

In [30]:
oeffentlich, privat = erzeuge_elgamal_schluesselpaar(p, g, x=10)
print(oeffentlich)
print(privat)

(23, 5, 9)
(23, 5, 10)


In [31]:
nachricht = 17
chiffrat = elgamal_verschluesseln(nachricht, oeffentlich)
chiffrat

(10, 19)

In [32]:
elgamal_entschluesseln(chiffrat, privat)

17